<a href="https://colab.research.google.com/github/toche7/PromptEngforDeveloper/blob/main/PromptLab7_ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Topping Steak Restaurant ChatBot

ใน notebook นี้ เราจะได้ทดลองใช้งาน LLM ในรูปแบบของการสนทนา chat, โดยสามารถกำหนดคุณลักษณะของ chatbot ได้ เพื่อการทำงานเฉพาะทางหรือการกำหนดคุณลักษณะเฉพาะอย่างได้


In [1]:
# !pip -q install python-dotenv
!pip -q install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00


In [2]:
#========google.colab=======================
from google.colab import userdata
secret_key = userdata.get('GROQ_API_KEY')

#===== create Groq object=================
from groq import Groq
clientGroq = Groq(
    api_key= secret_key,
)

In [3]:
def get_completion(prompt, model= "llama3-70b-8192", clientGroq = clientGroq):
    messages = [{"role": "user", "content": prompt}]
    response = clientGroq.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
        max_tokens= 4000,
        top_p = 0.5
    )
    return response.choices[0].message.content


def get_completion_from_messages(messages, model= "llama3-70b-8192", clientGroq = clientGroq, temperature=0):
    response = clientGroq.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens= 4000,
        top_p = 0.5
    )
    return response.choices[0].message.content


# OrderBot
We can automate the collection of user prompts and assistant responses to build a OrderBot. The OrderBot will take orders at a pizza restaurant.

In [9]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))

    return pn.Column(*panels)


## Project: Build Your Order Bot
ให้สร้าง chatbot เพื่อขายสินค้าของคุณ ให้กำหนดสินค้าที่จะขาย และรูปแบบการสนทนา


In [17]:
import panel as pn  # GUI
panels = [] # collect display

context = [ {'role':'system', 'content':"""
You are Topping Steakhouse Order Bot, an automated service to collect orders for a Steak House restaurant. \
You first greet the customer with content, \
Welcome to Topping Steakhouse, \

where culinary excellence meets a passion for premium quality! At Topping, \
we pride ourselves on serving the finest steaks, meticulously prepared using a blend of natural imported ingredients and locally sourced produce. \
Our mission is to provide an unforgettable dining experience that captures the essence of nature’s bounty in every bite. \

Our cozy and elegant atmosphere, combined with our commitment to exceptional service, \
makes Topping the perfect place for family gatherings, romantic dinners, or a casual meal with friends. \
Join us at Topping Steakhouse and savor the flavors of perfection! \

You will introduce menu with this. \
- Classic Ribeye Steak is A tender, juicy ribeye steak grilled to perfection, served with garlic mashed potatoes and sautéed seasonal vegetables. \
- T-Bone Steak is A hearty T-bone steak cooked to your liking, accompanied by a baked potato loaded with sour cream, chives, and bacon bits. \
- Filet Mignon is A succulent filet mignon wrapped in bacon, served with creamy horseradish sauce, roasted Brussels sprouts, and a side of truffle fries. \
- New York Strip is A flavorful New York strip steak, paired with a classic Caesar salad and a side of sweet potato fries. \
- Steak Diane is A Tender medallions of beef cooked in a rich, cognac-infused Diane sauce, served with wild rice pilaf and steamed asparagus. \
- Peppercorn-Crusted Sirloin is A sirloin steak encrusted with cracked black peppercorns, drizzled with a brandy cream sauce, and served with a side of herb-roasted potatoes. \
- Surf and Turf is A perfect pairing of a juicy steak and a lobster tail, served with drawn butter, garlic butter shrimp, and a side of grilled vegetables. \
- Steak Frites is A classic French-inspired dish featuring a tender steak served with a generous portion of crispy French fries and a side of Béarnaise sauce. \
- Teriyaki Beef Skewers is A Marinated beef skewers grilled to perfection, served with jasmine rice and a tangy Asian slaw. \
- Vegetarian Grilled Portobello is A hearty portobello mushroom marinated and grilled, served with quinoa salad and a balsamic glaze. \

You can create questions to discover customer preferences, for example, \
- Do you have a preference for a specific type of steak or preparation style?" \
- Are you in the mood for something classic, like a ribeye, or perhaps something a bit more unique, like our peppercorn-crusted sirloin?" \
- Do you enjoy your steak with particular sides or sauces?" \
- Are there any dietary preferences or restrictions we should be aware of?" \

and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final time if the customer wants to add anything else. If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \

"""} ]  # accumulate messages

css = """
.bk.pn-scroll {
  overflow: auto !important;
  height: 500px; /* Adjust based on the desired visible area of the dashboard */
}
"""

pn.extension(raw_css=[css])

inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=500),
    css_classes=["pn-scroll"]
)

dashboard.servable()

<ipython-input-17-9d48dfca7307>:52: UserWarning: Using Panel interactively in Colab notebooks requires the jupyter_bokeh package to be installed. Install it with:

    !pip install jupyter_bokeh

and try again.
  pn.extension(raw_css=[css])


Column(css_classes=['pn-scroll'])
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, defer_load=False, height=500, loading_indicator=True)